In [7]:
import numpy as np
import qiskit as qk
from qiskit import Aer
%matplotlib inline

In [8]:
def basis_encoding(x):
    M,N = x.shape
    clas_reg = qk.ClassicalRegister(N)
    loading_reg = qk.QuantumRegister(N, name = "loading")
    storage_reg = qk.QuantumRegister(N, name = "storage")
    ancillas = qk.QuantumRegister(N, name = "ancillas")
    branches = qk.QuantumRegister(2, name = "branches")
    
    
    circuit = qk.QuantumCircuit(clas_reg, loading_reg, storage_reg, ancillas, branches)
    circuit.x(branches[1])
    circuit.barrier()
    for i in range(M):
        for j in range(N):
            if x[i,j] == 1:
                circuit.x(loading_reg[j])
                circuit.cx(branches[1], storage_reg[j])
        
        circuit.barrier()
        
        circuit.cx(branches[1], branches[0])
        theta = 1/np.sqrt(M-i)
        circuit.cry(2*np.arcsin(theta), branches[0], branches[1])
        
        circuit.barrier()
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        circuit.mcx(ancillas,branches[0])
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        circuit.barrier()
        
        for j in range(N):
            if x[i,j] == 1:
                circuit.x(loading_reg[j])
                circuit.cx(branches[1], storage_reg[j])
        
        circuit.barrier()

    circuit.measure(storage_reg, clas_reg)
    return circuit

In [9]:
x = np.array([[1,0]])
backend = Aer.get_backend('qasm_simulator')

circuit = basis_encoding(x)
circuit.draw()

job = qk.execute(circuit, backend, shots=1000)
result = job.result()
print(result.get_counts(circuit))

{'01': 1000}
